# Single Activation Foil Counting Tools

This notebook is to aid in determining optimal count times given a set of activations and experiments design parameters. It requires PyNE to be loaded for nuclear data purposes. It is intended as a single use version.  

A batch version is contained in the batchActivationCountTime notebook.  

A more automated version for scripting and developing overall count plans can be found in the 

Import the following packages:

In [11]:
import sys
import os
import numpy as np

from pyne import data
from pyne import nucname
from scipy.integrate import quad
from math import exp, sqrt

# Path to support scripts 
sys.path.insert(0,os.path.abspath('/home/pyne-user/Dropbox/UCB/Computational_Tools/Scripts/Python/GeneralNuclear'))
from BasicNuclearCalcs import fractional_solid_angle, activity, production_decay
from Counting import volume_solid_angle, germanium_eff_exp

## User Experimental Input

Specify the experimental parameters determining the reaction rate.  This assumes that the T<sub>1/2</sub> is >> 1 seconds, which should be true for all practical experiments.

### Beam Variables
src = the neutron source strength in in n/sec

src_t = irradition time in s

In [15]:
src=4.93E9
t=3600*2

### Foil Variables
foil = name of the foil reaction product written in the format "XXAAA" - ex "U235" or "Rb86"

rx_rate = the reaction rate per source particle in units of reactions/cm<sup>3</sup>/src (this can be obtained from simulation or a simple calculation). It assumes that the natural abundance of the isotope is accounted for in this rate.  

foil_r = the foil radius in cm

foil_h = the foil height in cm

foil_rho = the foil density in g cm<sup>-3</sup>

vol = the foil volume in cm<sup>3</sup>

trans_t = the foil transfer time post-irradiation in sec

In [31]:
#Basic foil parameters
act_product="Zr89"
rx_rate=1.41E-6
foil_r=2.5
foil_h=0.1
foil_rho=6.49
BR=99.04
gamma_energy=909.15

#Delay between irradiation and counting
decay_t=103320

# Calculate the foil volume
vol=np.pi*foil_r**2*foil_h

# Calculate the initial number of atoms from the irradiation (assumes natural abundance is captured in rx_rate)
n0=production_decay(data.half_life(act_product), 0.0, t, rx_rate, src, vol, decay_t)

print "The number of {} atoms in the sample after irradiation and {} s decay= {:.3e}".format(act_product,decay_t,n0)

The number of Zr89 atoms in the sample after irradiation and 103320 s decay= 7.558e+07


### Decay Data

The following uses PyNE to get decay data for the foil.  This is used to get the branching ratio of the primary gamma and provide general information about the decay radiation.

Future revision can use this data to add automated functionality such as background count given the primary gamma.  

In [32]:
# Get level transitions
decay_pairs = data.gamma_from_to_byparent(nucname.id(act_product))

# Get gamma ray energies
energies = data.gamma_energy(nucname.id(act_product))

# Relative gamma intensities
intensities = data.gamma_photon_intensity(nucname.id(act_product))

# Converts the relative intensities to decays per 100 decays of the parent and print 
print "Parent (t_1/2 [s])    Daughter    Level Transition     Gamma (keV)           BR (%)"
print "======================================================================================="
photonbr, photonbr_error = data.decay_photon_branch_ratio(nucname.id(act_product),decay_pairs[0][1])
final_intensities = []
for i in range(len(intensities)):
    # compute the intensities by multiplying the branch ratio and the relative intensity; ignore the errors 
    final_intensities.append(photonbr*intensities[i][0])
    print "{} ({})        {:5s}           {}->{}           {} +/-{}         {}".format(act_product, data.half_life(act_product), 
                                            nucname.name(decay_pairs[i][1]), decay_pairs[i][0]%100, \
                                            decay_pairs[i][1]%100, energies[i][0], energies[i][1],\
                                            final_intensities[-1])
    
print "\n\n\nWARNING:BR and gamma energy must be specified by user. PyNE automated interfaces removed due to incomplete data."

Parent (t_1/2 [s])    Daughter    Level Transition     Gamma (keV)           BR (%)
Zr89 (282276.0)        Y89             1->0           909.15 +/-0.15         99.04
Zr89 (282276.0)        Y89M            5->1           1620.8 +/-0.2         0.0732896
Zr89 (282276.0)        Y89M            6->1           1657.3 +/-0.2         0.1059728
Zr89 (282276.0)        Y89M            7->1           1713.0 +/-0.6         0.7447808
Zr89 (282276.0)        Y89             3->0           1744.5 +/-0.2         0.1228096





### Counting Facility Variables
background = the background rate at the peak of interest in counts/s

det_r = radius of the detector in cm

det_foil_dist = the distance from the detector face to the foil in cm

sigma = the desired counting statistics level

In [33]:
background=0.01
det_r=3.245
det2foil_dist=1
sigma=0.01

### Calculate the absolute efficiency

Takes into account the geometry correction factor (gcf), detector intrinsic efficiency, and the energy and position dependence of those variables. Change the foil distance if significant dead time encountered.

In [34]:
while True:
    # Calculate the GCF
    gcf=volume_solid_angle(foil_r,det_r,det2foil_dist)

    # Calculate absolute efficiency
    abs_eff=germanium_eff_exp(gamma_energy, a=4.36039089e-06, b=2.11739894e+00, c=5.40245119e-01, d=5.92997979e-01) \
            *(gcf/fractional_solid_angle(det_r,det2foil_dist))

    # Use a simple paralyzable model to calculate worst case scenario
    n=activity(data.half_life(act_product),n0)*abs_eff*BR/100
    tau=1E-5
    meas=n*exp(-n*tau)

    if n/meas>1.01:
        det2foil_dist+=1
    else:
        break
        
    if det2foil_dist>5:
        print "ERROR: The foil is hot!"
        break

print "The foil is {} cm from the detector, and the dead time is {}%".format(det2foil_dist,(n/meas-1)*100)
print "The GCF is {}, and the pt src fractional solid angle is {}.".format(gcf,fractional_solid_angle(det_r,det2foil_dist))
print "The overall absolute efficiency is {:2f}% @ {} keV".format(abs_eff*100,gamma_energy)

The foil is 1 cm from the detector, and the dead time is 0.00425569693876%
The GCF is 0.316168194466, and the pt src fractional solid angle is 0.352750143624.
The overall absolute efficiency is 2.315202% @ 909.15 keV


### Counting time calculations

Bringing it all together....

In [35]:
# Define the activity integrand accounting for all of the efficiencies
def integrand(t):
    return activity(data.half_life(act_product),n0,t)*abs_eff*BR/100
    
# Report the starting activities
print "The activity of {} for the {} line at the start of counting ({} s after irradiation) = {}"\
       .format(act_product,gamma_energy,decay_t,activity(data.half_life(act_product),n0)*BR/100)
print "The specific activity of {} for the {} line at the start of counting ({} s after irradiation) = {}"\
       .format(act_product,gamma_energy,decay_t,activity(data.half_life(act_product),n0)*BR/100/(vol*foil_rho))

# Approximate the optimal foil counting time using an average count rate
tf=1
diff=1000
try:
    while diff > 1:
        prevt=tf
        S = quad(integrand, 0, tf)[0]/tf
        tf=((sqrt(S+background)+sqrt(background))**2/(sigma**2*S**2))/(1+1/sqrt((S+background)/background))  #Knoll eqn 3.54/55
        diff=tf-prevt
    print "The average count rate in the detector over the counting period is {:.3f}".format(S)
    print "The optimal count time for the {} foil with a desired statistical level of sigma={} is {:.0f} sec [{:.2f} hr]."\
          .format(act_product,sigma,tf,tf/3600)
    # Calculate the optimal time for background counting
    tb=tf/sqrt((S+background)/background)
    print "The optimal time for background counting is {:.0f} sec [{:.2f} hr]".format(tb,tb/3600)
except ZeroDivisionError:
    tf=1E99
    print "{}% statistics cannot be achieved with this setup".format(sigma*100)


The activity of Zr89 for the 909.15 line at the start of counting (103320 s after irradiation) = 183.81146983
The specific activity of Zr89 for the 909.15 line at the start of counting (103320 s after irradiation) = 14.424408762
The average count rate in the detector over the counting period is 4.243
The optimal count time for the Zr89 foil with a desired statistical level of sigma=0.01 is 2477 sec [0.69 hr].
The optimal time for background counting is 120 sec [0.03 hr]


In [51]:
print S, background, sigma
print ((sqrt(S+background)+sqrt(background))**2/(sigma**2*S**2))/(1+1/sqrt((S+background)/background))

8.33407394595 0.01 0.01
1242.92171225
